In [8]:
import re
import subprocess
import numpy as np
import pickle
class lkh:
    def __init__(self , dist_mat_pic_file):
        self.dist_mat_pic_file = dist_mat_pic_file
        self.dist_mat_file_name = dist_mat_pic_file.split(".")[0]
        print(self.dist_mat_file_name)

    def solve(self):
        '''
        solves the TSP instance using .exe file
        '''
        # Generate the .par file
        self.generate_parameter_file(self.dist_mat_file_name)

        # Generate the .atsp file
        dist_mat = self.read_pickle(self.dist_mat_pic_file)
        self.generate_atsp_file(dist_mat , self.dist_mat_file_name)

        try :    
            # Replace "E3k.0.par" with the actual parameter for your .exe file
            result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par", "-S" , f"{self.dist_mat_file_name}_start.txt" ], stdin=subprocess.DEVNULL , timeout = 300)
            #result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par", "-S" , f"{self.dist_mat_file_name}_start.txt" ], stdin=subprocess.DEVNULL , timeout = 300)
        
        except subprocess.TimeoutExpired:
            print("Execution timed out. Terminating the process.")
            # print(output)
        
        result_file_name = self.dist_mat_file_name + '.txt'
        print("Result is saved in : ",result_file_name)
        length , tour_section = self.get_obj_val_and_tour(result_file_name)

        print("Objective function value : ",length)
        print("The routing : " , tour_section)

    
    def get_obj_val_and_tour(self , tour_file):
        """
        Extracts the length and tour section from a tour file.

        Args:
            tour_file (str): Path to the tour file.

        Returns:
            tuple: A tuple containing the length and tour section as a list.
        """
        length = None
        tour_section = []

        with open(tour_file, "r") as file:
            for line in file:
                line = line.strip()

                if line.startswith("COMMENT : Length"):
                    length = int(line.split("=")[1].strip())
                elif line == "TOUR_SECTION":
                    # Start reading the tour section
                    for tour_line in file:
                        tour_line = tour_line.strip()
                        if tour_line == "-1":
                            # End of tour section
                            break
                        tour_section.append(int(tour_line))

        return length, tour_section


    def get_obj_val(self , output):
        '''
        returns the objective value from the  long output value
        '''
        # Assuming `output` contains the captured output
        cost_min_match = re.search(r'Cost\.min = (\d+)', output)

        if cost_min_match:
            cost_min_value = int(cost_min_match.group(1))
            return cost_min_value
        else:
            print("Cost.min not found in the output.")
            return 0    
    
    def get_routing(self , filename):
        '''
        returns the routing in a numpy array
        '''
        start_keyword = 'TOUR_SECTION'
        end_keyword = '-1'
        with open(filename, 'r') as file:
            lines = []
            read_flag = False
            for line in file:
                if start_keyword in line:
                    read_flag = True
                    continue  # skip this line
                if read_flag:
                        if end_keyword in line:
                            read_flag = False
                            break
                        else:
                            lines.append(line.strip())

        lines = np.array(lines)
        return lines
    
    def generate_parameter_file(self , filename):
        runs = 1
        filename_atsp = filename + ".atsp"
        filename_txt = filename + ".txt"  
        filename_par = filename + ".par"

        with open(filename_par, 'w') as f:
            f.write(f"PROBLEM_FILE = {filename_atsp}\n")
            f.write(f"OUTPUT_TOUR_FILE = {filename_txt}\n")
            f.write(f"OPTIMUM = 40634081 \n")
            f.write(f"INITIAL_PERIOD = 1000 \n")
            f.write(f"MAX_CANDIDATES = 6 \n")
            f.write(f"MAX_TRIALS = 1000 \n")
            f.write(f"MOVE_TYPE = 6 \n")
            f.write(f"PATCHING_C = 6 \n")
            f.write(f"PATCHING_A = 5 \n")
            f.write(f"RECOMBINATION = GPX2 \n")
            f.write(f"RUNS = {runs} \n")

    
    def round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to round up numbers and return as integers
        '''
        rounded_lst = []
        for inner_lst in lst:
            rounded_inner_lst = []
            for num in inner_lst:
                rounded_num = int(round(num))
                rounded_inner_lst.append(rounded_num)
            rounded_lst.append(rounded_inner_lst)
        return rounded_lst  


    def read_pickle(self,pickle_path):
        # Open the pickle file in read-binary mode
        with open(pickle_path, 'rb') as f:
            # Load the object from the pickle file
            pic_obj = pickle.load(f)
        
        pic_obj = self.round_up(pic_obj)


        # Do something with the object
        print(pic_obj)
        
        return pic_obj
    
    def generate_atsp_file(self , dist_mat, filename):   
        '''
        input : 
        a symmentric matrix  
        directory of a .tsp file you want to generate
        number of cities in the TSP

        Output : 
        a .tsp file generated in the filename directory
        '''
        # Defining the number of cities
        num_cities = len(dist_mat)

        # Write the coordinates to a .atsp file
        # fname = filename.split("/")[-1].split(".")[0]
        fname = filename 
        with open(filename+".atsp", 'w') as f:
            f.write(f"NAME: {fname}\n")
            # f.write("COMMENT: Randomly generated TSP instance\n")
            f.write("TYPE: TSP (M.~Hofmeister)\n")
            f.write(f"DIMENSION: {num_cities}\n") #becase the distance matrix doubles in size when converted to a symmentric instance
            f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
            f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
            # f.write("NODE_COORD_SECTION\n")
            f.write("DISPLAY_DATA_TYPE: NO_DISPLAY\n")
            f.write("EDGE_WEIGHT_SECTION\n")
            for i in range(len(dist_mat)):
                for j in range(len(dist_mat)):
                    f.write(f'{str(dist_mat[i][j])}\t')
                f.write("\n")
            f.write("EOF\n")

    

    

s1 = lkh("dm_Warmsen_Saturday_31600-01_scenario_high_all.p")
s1.solve()
        

dm_Warmsen_Saturday_31600-01_scenario_high_all
[[0, 4839, 8242, 8269, 4830, 1876, 4848, 4863, 4874, 4885, 4891, 4901, 4904, 5419, 5507, 5481, 5470, 5439, 5431, 5425, 5413, 5402, 5384, 5366, 5352, 5341, 7797, 7663, 7502, 7485, 7468, 7453, 7441, 7428, 5321, 5323, 5167, 6158, 6064, 5914, 5900, 5807, 5796, 5790, 5874, 6084, 6235, 6071, 6043, 5931, 1739, 1684, 1670, 1663, 1526, 1519, 1516, 1477, 8106, 8133, 1725, 1735, 5821, 9169, 8144, 8178, 8183, 2066, 2068, 2072, 2075, 2083, 2098, 2133, 2150, 2166, 2188, 2201, 2213, 2223, 2238, 2215, 2205, 2194, 222, 218, 206, 193, 183, 179, 5393, 5424, 5409, 5386, 4902, 4912, 4928, 4966, 9435, 9426, 9347, 9350, 9354, 9358, 9365, 9371, 6024, 6043, 11548, 11593, 9382, 9391, 9332, 9339, 9343, 9402, 9411, 9416, 9423, 9428, 9444, 9462, 9466, 9472, 9462, 9456, 9452, 9448, 9442, 9438, 9416, 9401, 9394, 9388, 9384, 9379, 9375, 2184, 2134, 2125, 2120, 1990, 1995, 2013, 2020, 2046, 2060, 2068, 2076, 2082, 2107, 1994, 1990, 1963, 1958, 2051, 2034, 2003, 1992, 1980

#85790